<h1> Price Prediction for Cars </h1>
using given sample data

* Step 1: testing if python setup for jupiter notebook is working

In [1]:
print("hello world")

hello world


* Step 2: install the libraries

In [2]:
! pip install boto3
! pip install pandas
! pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 KB 377.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 KB 723.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 2.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.34
    Not uninstalling botocore at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'botocore'. No files were found to uninstall.
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.0
    Not uninstalling s3transfer at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 's3transfer'. No files were found to uninstall.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 328.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 407.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 499.5 k

In [ ]:
! pip install joblib

* Step 3: visualize the table

In [ ]:
import pandas as pd

df = pd.read_csv('data/sample_data.csv')
df

* Step 4: Remove Duplicate records

in our sample there are no duplicate records. however keeping the cleanup step will help future datasets

In [ ]:
# check how many duplicates exist
duplicates = df[df.duplicated(keep=False)]
print(duplicates.to_string(index=False))

df = df.drop_duplicates()

* Step 5: eleminate records with NaN/Null values

In general, if the NaN's are derivable, they should be derived. 

Note: But beaware that unknown value is not zero. that may express better results but that would corrupt the model to unrealisitic training.

In [ ]:
selected_columns = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'carheight', 'curbweight', 'carlength', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'Price']

df = df.drop_duplicates(selected_columns, keep=False)
df

null_columns = df.columns[df.isnull().any()].tolist()
null_columns

columns_check = ['CarName', 'fueltype',
 'carbody',
 'enginelocation',
 'carlength',
 'cylindernumber',
 'horsepower']

rows_with_nulls = df[df[columns_check].isnull().any(axis=1)]
rows_with_nulls

df = df.dropna(subset=selected_columns)
df

rows_with_nulls = df[df[columns_check].isnull().any(axis=1)]
rows_with_nulls[columns_check]

* Step 6: ensure their are no corruption in data and if they are try to fix them or eleminate them from the sample.
* Step 7: remove the columns that are unwanted.

In [ ]:
# Cleaning data
selected_columns = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'Price']
# df['carsize'] = df['carheight'] * df['curbweight'] * df['enginesize']

df = df.drop_duplicates(selected_columns)
tiny_df = df[selected_columns]
filtered_df = tiny_df[tiny_df[selected_columns].notna()]

# filtered_df.head()
sorted_df = filtered_df.sort_values(by='Price', ascending=True)
#print(sorted_df);
sorted_df.head()


<h2> Training the model </h2>
* This is typical problem of prediction of a numerical value based on given parameter.
* The model is basically a curve fitting problem where x are given properties and y is a point we are interested interested in.

Regression Linear/Curve[non linear]

NOTE: Our dataset is very small therefore Neural Networks wont be good approach. 
1.  define features, target, identify categorical feature and numeric features.
2.  build the pipelines
3.  split the dataset into Training and Testing dataset. 
4.  execute training process (fit)
5.  execute the Test and evaluate the performance. (predict for the known targets)
6.  execute the Predict on real world data to predict the Price

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

features = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']
target = 'Price'

x = df[features]
y = df[target]

categorical_features = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation']
numeric_features = ['carheight', 'curbweight', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']

# Imputers for column transformation
categorical_imputer = SimpleImputer(strategy='most_frequent')
numeric_imputer = SimpleImputer(strategy='mean')
preprocessor = ColumnTransformer(transformers=[
    ('cat', Pipeline([
        ('imputer', categorical_imputer),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features),
    ('num', numeric_imputer, numeric_features)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # ('regressor', LinearRegression())
    ('RandomForest', RandomForestRegressor(random_state=42))
])

<h2> Training the model </h2>

In [ ]:
# Train-test split and train
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
pipeline.fit(x_train, y_train)

In [ ]:
import joblib
joblib.dump(pipeline, "model.priceprediction.pkl")


<h2> Performance Evaluation </h2>
The model is not performing very well. the Mean Squared Error is significantly high. 
Mean Squared Error: 7147802.95 is significanly high value.
apparently the dataset is of large range too


In [ ]:
import numpy as np

# Predict and evaluate
y_pred = pipeline.predict(x_test)
mse = mean_squared_error(y_test, y_pred)

# Output results
print("Predicted Prices:", y_pred)
print("Mean Squared Error:", mse)

# Error analysis
y_test = np.array(y_test)
y_pred = np.array(y_pred)

diff  = y_test - y_pred
print(f" mean = {diff.mean()} \n max= {diff.max()} \n min = {diff.min()} \n standard deviation = {diff.std()}")
